# Scraping the class and year information of patents from the US Patent and Trademark Office website

In [12]:
import urllib2
import pandas as pd
from bs4 import BeautifulSoup
from mechanize import Browser

In [13]:
patents = pd.read_excel("/Users/satrajitmaitra/WebScraping/data/Patents.xlsx")
df = pd.DataFrame(patents)

In [14]:
pi_list = []
for index, row in df.iterrows():
    pi_list.append([row['Patent'], row['Inventor_Number']])

# Request the url for all the patent numbers in the dataframe and extract class and year information

In [16]:
browser = Browser()
class_list = []
year_list = []
plist = [pi_list[i][0] for i in range(len(pi_list))]
plist = (list(set(plist)))

for i in range(len(plist)):
    print plist[i], i
    browser.set_handle_robots(False)
    browser.set_handle_robots(False)
    browser.addheaders = [('User-agent','Mozilla/5.0 (X11; Linux x86_64; rv:18.0)Gecko/20100101 Firefox/18.0 (compatible;)'),('Accept', '*/*')]

    browser.open("http://patft.uspto.gov/netahtml/PTO/search-bool.html").read()

    if len(browser.forms()) != 0:
        browser.select_form(nr=0)

        browser['TERM1'] = str(plist[i])
        browser['FIELD1'] = ['PN']
        response = browser.submit()

        content = response.read()

        soup = BeautifulSoup(content)
        info = soup.select('table')
        if len(info) > 5:
            pnos = info[5].findAll('td')[1].text.split('; ')
            patent_class = ''
            for no in pnos:
                patent_class += no.split('/')[0]+'/'
            print patent_class
            class_list.append(patent_class)
        else:
            class_list.append("Cannot extract")
        if len(info) > 3:
            date = info[3].findAll('b')[-1].text.split(', ')
            if len(date) > 1:
                year_list.append(date[1])
                print date[1]
            else:
                year_list.append('Cannot extract')
        else:
            year_list.append('Cannot extract')

    

6295557 0
709/709/709/714/
5742604 1
370/370/
6600724 2

August 1984/
1998
5632021 3

November 1994/
1995
6205465 4

August 1998/
1998
5974051 5
370/370/710/
6205477 6

June 1999/
1998
6078504 7

November 1990/
1997
6049833 8

October 1985/
1997
6049834 9

March 1994/
1998
6115374 10

December 1984/
1995
6205487 11

/
6424624 12
370/370/370/709/
1998
6088754 13

August 1986/
1997
6285364 14

September 1998/
1997
6366558 15

/
5742649 16

October 1990/
1995
5802042 17

April 1992/
1996
6256699 18

February 1991/
1998
5854901 19

September 1992/
1996
7233600 20

December 1979/
1998
6339595 21

September 1993/
1997
6473798 22

October 1977/
1998
6424649 23
6381646 24
6385747 25
6377577 26
6418543 27
7701924 28
6101115 29
6314110 30
6512769 31
6058114 32
6192491 33
6058117 34
D0431557 35
5884043 36
6115468 37
5838994 38
5970067 39
6181572 40
6408347 41
6178160 42
6138019 43
7020712 44
6052012 45
5834925 46
6101169 47
6490290 48
6523060 49
6195385 50
5905781 51
6775840 52
5793987 53
6240461

# Create a coherrent list having all the information

In [ ]:
combined_list = []
for i in range(len(plist)):
    for j in range(len(pi_list)):
        if plist[i] == pi_list[j][0]:
            combined_list.append([plist[i], pi_list[j][1], year_list[i], class_list[i]])

In [229]:
combined_list

[[6295557, 348, 'Cannot extract', u'709/709/709/714/'],
 [6295557, 751, 'Cannot extract', u'709/709/709/714/'],
 [6295557, 752, 'Cannot extract', u'709/709/709/714/'],
 [6295557, 1433, 'Cannot extract', u'709/709/709/714/'],
 [5742604, 497, 'Cannot extract', u'370/370/'],
 [5742604, 2208, 'Cannot extract', u'370/370/'],
 [6600724, 1404, u'1998', u'\nAugust 1984/'],
 [5632021, 434, u'1995', u'\nNovember 1994/'],
 [5632021, 435, u'1995', u'\nNovember 1994/'],
 [5632021, 436, u'1995', u'\nNovember 1994/'],
 [6205465, 46, u'1998', u'\nAugust 1998/'],
 [6205465, 986, u'1998', u'\nAugust 1998/'],
 [6205465, 987, u'1998', u'\nAugust 1998/'],
 [5974051, 713, 'Cannot extract', u'370/370/710/'],
 [5974051, 1083, 'Cannot extract', u'370/370/710/'],
 [6205477, 989, u'1998', u'\nJune 1999/'],
 [6205477, 990, u'1998', u'\nJune 1999/'],
 [6205477, 991, u'1998', u'\nJune 1999/'],
 [6078504, 78, u'1997', u'\nNovember 1990/'],
 [6049833, 803, u'1997', u'\nOctober 1985/'],
 [6049833, 804, u'1997', u'\nOc

In [235]:
headers = ['Patent Number', 'Inventor Number', 'Year', 'Class']
df1 = pd.DataFrame(combined_list, columns = headers)

In [237]:
df1.to_csv('/Users/satrajitmaitra/Desktop/Patents_Scraping_Output.csv', sep='\t', encoding='utf-8')